## About
This notebook has been forked from [Shopee: Siamese ResNet-50 with triplet loss on TPU](https://www.kaggle.com/xhlulu/shopee-siamese-resnet-50-with-triplet-loss-on-tpu), so please go and check out the notebook and support his work.<br>
I have tried to solve the Shopee similar image estimation problem with reference to the research paper [Retrieving Similar E-Commerce Images Using Deep Learning](https://arxiv.org/pdf/1901.03546v1.pdf), code - https://github.com/gofynd/mildnet

In [ ]:
!pip install keras-toolkit==0.1.0rc3 -q

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt

from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import vgg16
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

import keras_toolkit as kt
from kaggle_datasets import KaggleDatasets
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# bokeh packages
from bokeh.io import output_file,show,output_notebook,push_notebook
from bokeh.plotting import *
from bokeh.models import ColumnDataSource,HoverTool,CategoricalColorMapper
from bokeh.layouts import row,column,gridplot,widgetbox
from bokeh.models.widgets import Tabs,Panel
output_notebook()

In [ ]:
seed=42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(123)

In [ ]:
target_shape = (224, 224)
def preprocess_image(filename, target_shape=target_shape):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )


In [ ]:
COMPETITION_NAME = 'shopee-product-matching'
strategy = kt.accelerator.auto_select(verbose=True)
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)
BATCH_SIZE = strategy.num_replicas_in_sync * 16

In [ ]:
train = pd.read_csv('../input/shopee-generate-data-for-triplet-loss/train_triplets_imgs.csv')

train = train.apply(lambda col: GCS_DS_PATH + '/train_images/' + col)
train_paths, val_paths = train_test_split(train, train_size=0.8, random_state=42)
train_paths.head()

In [ ]:
dtrain = kt.image.build_dataset(
    (train_paths.anchor, train_paths.positive, train_paths.negative), 
    decode_fn=preprocess_triplets,
    bsize=BATCH_SIZE,
    augment=False,
    repeat=False
)

dvalid = kt.image.build_dataset(
    (val_paths.anchor, val_paths.positive, val_paths.negative), 
    decode_fn=preprocess_triplets,
    bsize=BATCH_SIZE,
    augment=False,
    repeat=False
)

## Visualize data

In [ ]:
def visualize(pairs):
    anchor, positive, negative = pairs
    fig, ax = plt.subplots(10,3, figsize=(10,20))
    
    for i in range(10):
        ax[i, 0].imshow((anchor[i]))
        ax[i, 1].imshow((positive[i]))
        ax[i, 2].imshow((negative[i]))
        ax[i, 0].set_title("anchor")
        ax[i, 1].set_title("Positive")
        ax[i, 2].set_title("Negative")
        ax[i, 0].axis('off')
        ax[i, 1].axis('off')
        ax[i, 2].axis('off')

In [ ]:
visualize_samples = next(iter(dtrain))
visualize(visualize_samples)

In [ ]:
visualize_samples = next(iter(dtrain))
visualize(visualize_samples)

In [ ]:
'''with strategy.scope():
    base_cnn = resnet.ResNet50(
        weights="imagenet", input_shape=target_shape + (3,), include_top=False
    )

    flatten = layers.Flatten()(base_cnn.output)
    dense1 = layers.Dense(512, activation="relu")(flatten)
    dense1 = layers.BatchNormalization()(dense1)
    dense2 = layers.Dense(256, activation="relu")(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    output = layers.Dense(256)(dense2)

    embedding = Model(base_cnn.input, output, name="Embedding")

    trainable = False
    for layer in base_cnn.layers:
        if layer.name == "conv5_block1_out":
            trainable = True
        layer.trainable = trainable'''

In [ ]:
def get_layers_output_by_name(model,layer_names):
        return {v: model.get_layer(v).output for v in layer_names}

## Note
The commented segment shown in the below code cell represents the **Multi-scale convolutional neural network** architecture as discussed in the research paper [Retrieving Similar E-Commerce Images Using Deep Learning](https://arxiv.org/pdf/1901.03546v1.pdf), which I tried to build but was facing errors when I was trying to train the model. **SiameseModelExperiment** is the training model class that I built for the model if **Multi-scale convolutional neural network** was the architecture.<br>
Please give it try to solve the error and I wish you could solve it.

In [ ]:
with strategy.scope():
    vgg_model = vgg16.VGG16(weights="imagenet", include_top=False, input_shape=target_shape + (3,))
    for layer in vgg_model.layers[:10]:
            layer.trainable = False
    intermediate_layer_outputs = get_layers_output_by_name(vgg_model, 
                                                        ["block1_pool", "block2_pool", "block3_pool", "block4_pool"])
    convnet_output = layers.GlobalAveragePooling2D()(vgg_model.output)
    for layer_name, output in intermediate_layer_outputs.items():
            output = layers.GlobalAveragePooling2D()(output)
            convnet_output = layers.concatenate([convnet_output, output])
    convnet_output = layers.Dense(512, activation = 'relu')(convnet_output)
    convnet_output = layers.Dropout(0.6)(convnet_output)
    convnet_output = layers.Dense(512, activation = 'relu')(convnet_output)
    #convnet_output = layers.Dropout(0.5)(convnet_output)
    convnet_output = layers.Lambda(lambda p: K.l2_normalize(p,axis=1))(convnet_output)
    
    '''anchor_input = layers.Input(shape=target_shape + (3,),name="anchor")    
    s1 = layers.MaxPool2D(pool_size=(4,4),strides = (4,4),padding='valid')(anchor_input)
    s1 = layers.ZeroPadding2D(padding=(4, 4), data_format=None)(s1)
    s1 = layers.Conv2D(96, kernel_size=(8, 8),strides=(4,4), padding='valid')(s1)
    s1 = layers.ZeroPadding2D(padding=(2, 2), data_format=None)(s1)
    s1 = layers.MaxPool2D(pool_size=(7,7),strides = (4,4),padding='valid')(s1)
    s1 = layers.Flatten()(s1)
    
    positive_input = layers.Input(shape=target_shape + (3,),name="positive")    
    s2 = layers.MaxPool2D(pool_size=(8,8),strides = (8,8),padding='valid')(positive_input)
    s2 = layers.ZeroPadding2D(padding=(4, 4), data_format=None)(s2)
    s2 = layers.Conv2D(96, kernel_size=(8, 8),strides=(4,4), padding='valid')(s2)
    s2 = layers.ZeroPadding2D(padding=(1, 1), data_format=None)(s2)
    s2 = layers.MaxPool2D(pool_size=(3,3),strides = (2,2),padding='valid')(s2)
    s2 = layers.Flatten()(s2)'''
    
    '''merge_one = layers.concatenate([s1, s2])
    merge_one_norm = layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(merge_one)
    merge_two = layers.concatenate([merge_one_norm, convnet_output], axis=1)
    emb = layers.Dense(512)(merge_two)
    l2_norm_final = layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)'''
    
    #embedding= Model(inputs=[anchor_input, positive_input, vgg_model.input], outputs=convnet_output)
    embedding= Model(inputs=[vgg_model.input], outputs=convnet_output,name="Embedding")

In [ ]:
model =  embedding
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


with strategy.scope():
    anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
    positive_input = layers.Input(name="positive", shape=target_shape + (3,))
    negative_input = layers.Input(name="negative", shape=target_shape + (3,))

    distances = DistanceLayer()(
        embedding(vgg16.preprocess_input(anchor_input)),
        embedding(vgg16.preprocess_input(positive_input)),
        embedding(vgg16.preprocess_input(negative_input)),
    )

    siamese_network = Model(
        inputs=[anchor_input, positive_input, negative_input], outputs=distances
    )

In [ ]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The Contrastive Loss is defined as:
       L(θ) = (1-y)(1/2)D(Xq, Xp)^2 + y(1/2){max(0, m - D(Xq, Xn)^2)}
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        loss = tf.convert_to_tensor(0,dtype=tf.float32)
        g = tf.constant(1.0, shape=[1], dtype=tf.float32)
        h = tf.constant(0.0, shape=[1], dtype=tf.float32)
        
        def _contrastive_loss(y_true, y_pred):
            return tfa.losses.contrastive_loss(y_true, y_pred)
        
        ap_distance, an_distance = self.siamese_network(data)
        loss_query_pos = _contrastive_loss(g, ap_distance)
        loss_query_neg = _contrastive_loss(h, an_distance)
        loss = loss + loss_query_pos + loss_query_neg

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        #loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

## Training Model class for Multi-scale convolutional neural network architecture

In [ ]:
class SiameseModelExperiment(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.
    """

    def __init__(self, network, margin=0.5, batch_size=32):
        super(SiameseModelExperiment, self).__init__()
        self.model = network
        self.margin = margin
        self.batch_size = batch_size
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.model(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            q_emd = self.model((data[0], data[0], data[0]), training = True)
            p_emd = self.model((data[1], data[1], data[1]), training = True)
            n_emd = self.model((data[2], data[2], data[2]), training = True)
            loss_value = self.contrastive_loss_function(q_emd, p_emd, n_emd, self.batch_size)
            accuracy_value = self.accuracy(q_emd, p_emd, n_emd, self.batch_size)

        grads = tape.gradient(loss_value, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        return {"loss": loss_value, "acc":accuracy_value}
    
    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
    
    def contrastive_loss_function(self, q_emd, p_emd, n_emd, batch_size):
        '''
          Ref: https://github.com/gofynd/mildnet/blob/master/trainer/loss.py
          This function takes embedding generated by model for each of the image
          part of the triplet and return the loss value for the batch.
          q_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          p_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          n_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          batch_size(input): batch size for each step
          loss(output): Final loss for a batch
        '''
        def _contrastive_loss(y_true, y_pred):
            return tfa.losses.contrastive_loss(y_true, y_pred)

        loss = tf.convert_to_tensor(0,dtype=tf.float32)
        g = tf.constant(1.0, shape=[1], dtype=tf.float32)
        h = tf.constant(0.0, shape=[1], dtype=tf.float32)

        for obs_num in range(batch_size):
            dist_query_pos = tf.sqrt(tf.reduce_sum((q_emd[obs_num] - p_emd[obs_num])**2))
            dist_query_neg = tf.sqrt(tf.reduce_sum((q_emd[obs_num] - n_emd[obs_num])**2))
            loss_query_pos = _contrastive_loss(g, dist_query_pos)
            loss_query_neg = _contrastive_loss(h, dist_query_neg)
            loss = loss + loss_query_pos + loss_query_neg

        loss = loss/(batch_size*2)
        zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
        return tf.maximum(loss, zero)
    
    def accuracy(self, q_emd, p_emd, n_emd, batch_size):
        '''
          Ref: https://github.com/gofynd/mildnet/blob/master/trainer/accuracy.py
          This function takes in embedding and return the accuracy value for the batch
          q_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          p_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          n_emd(input): embedding generated by model for query image(tensor of size [batch, 4096])
          batch_size(input): batch size for each step
          accuracy(output): Final accuracy value for a batch
        '''
        accuracy = 0
        for obs_num in range(batch_size):
            dist_query_pos = tf.sqrt(tf.reduce_sum((q_emd[obs_num] - p_emd[obs_num])**2))
            dist_query_neg = tf.sqrt(tf.reduce_sum((q_emd[obs_num] - n_emd[obs_num])**2))
            accuracy += tf.cond(dist_query_neg > dist_query_pos, lambda : 1, lambda : 0)

        return (accuracy * 100) / batch_size

## Train

In [ ]:
with strategy.scope():
    siamese_model = SiameseModel(siamese_network)
    #siamese_model.compile(optimizer=optimizers.Adam(0.0001))
    siamese_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate = 0.001))

hist = siamese_model.fit(dtrain, epochs=20, validation_data=dvalid)

In [ ]:
siamese_model.save_weights('siamese_model.h5')
siamese_model.siamese_network.save_weights('siamese_network.h5')
embedding_net = siamese_model.siamese_network.get_layer("Embedding")
embedding_net.save_weights('embedding_net.h5')

## Visualization over validation data

In [ ]:
import plotly.express as px
px.line(hist.history)

In [ ]:
def visualize_test(pairs, pred):
    anchor, positive, negative = pairs
    fig, ax = plt.subplots(10,3, figsize=(10,20))
    
    for i in range(10):
        ax[i, 0].imshow((anchor[i]))
        ax[i, 1].imshow((positive[i]))
        ax[i, 2].imshow((negative[i]))
        ax[i, 0].set_title("anchor")
        ax[i, 1].set_title("Positive: {:.3f}".format(1 - pred[0][i]))
        ax[i, 2].set_title("Negative: {:.3f}".format(1 - pred[1][i]))
        ax[i, 0].axis('off')
        ax[i, 1].axis('off')
        ax[i, 2].axis('off')

In [ ]:
data_test = next(iter(dvalid))
pred = siamese_model.predict([data_test[0],data_test[1],data_test[2]])
visualize_test(data_test, pred)

In [ ]:
data_test = next(iter(dvalid))
pred = siamese_model.predict([data_test[0],data_test[1],data_test[2]])
visualize_test(data_test, pred)

In [ ]:
data_test = next(iter(dvalid))
pred = siamese_model.predict([data_test[0],data_test[1],data_test[2]])
visualize_test(data_test, pred)

In [ ]:
data_test = next(iter(dvalid))
pred = siamese_model.predict([data_test[0],data_test[1],data_test[2]])
visualize_test(data_test, pred)

## Extra Visualizations

In [ ]:
data_valid = pd.read_csv('../input/shopee-generate-data-for-triplet-loss/train_triplets_titles.csv')
data_valid =data_valid.loc[val_paths.index,:].reset_index(drop=True)

In [ ]:
data_valid2 =val_paths.reset_index(drop=True)

In [ ]:
pred = model.predict(dvalid)
pred = np.nan_to_num(pred)

In [ ]:
# Encoding all the images for inclusion in a dataframe.

from io import BytesIO
from PIL import Image
import base64

def embeddable_image(img):
    img = img.split('/')[-1]
    img = '../input/shopee-product-matching/train_images/'+img
    
    data = Image.open(img)
    data =np.asarray(data)
    
    img_data = data.astype(np.uint8)
    image = Image.fromarray(img_data).resize((50,50), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns

In [ ]:
scale = preprocessing.StandardScaler()
X = scale.fit_transform(pred)
X_norm = preprocessing.normalize(X)

pca_N = PCA(n_components=2)
principalComponents_N = pca_N.fit_transform(X_norm)

In [ ]:
# initial source
source = ColumnDataSource(data=dict(
    x = principalComponents_N[:, 0],
    y = principalComponents_N[:, 1],
    x_backup = principalComponents_N[:, 0],
    y_backup = principalComponents_N[:, 1],
    anchor = data_valid.anchor,
    positive = data_valid.positive,
    negative = data_valid.negative
))

# hover tool
hover = HoverTool(tooltips = [("anchor","@anchor"),("positive","@positive"),("negative","@negative")],
                 point_policy="follow_mouse")

# plotting
plot=figure(title ="PCA Visualization",
            plot_width=900, plot_height=850, 
            tools=[hover,"crosshair","pan","box_zoom","wheel_zoom", "reset"])
# plot settings
plot.scatter('x', 'y', size=5, 
          source=source,
          
          line_alpha=0.3,
          line_color="black")

# this is different from what we learn up to now.
# update method: When slider is changed or when different value from drop down tool is chosen this method is called.
# In this method x and y axis are updated from drop dawn value and year is updated from slider value.
show(plot,notebook_handle=True)

In [ ]:
# initial source
source = ColumnDataSource(data=dict(
    x = principalComponents_N[:, 0],
    y = principalComponents_N[:, 1],
    x_backup = principalComponents_N[:, 0],
    y_backup = principalComponents_N[:, 1],
    anchor = data_valid.anchor,
    image = list(map(embeddable_image, data_valid2.anchor))
))

# hover tool
#hover = HoverTool(tooltips = [("anchor","@anchor"),("positive","@positive"),("negative","@negative")],
#                 point_policy="follow_mouse")

# plotting
plot=figure(title ="PCA Visualization with Anchor Images",
            plot_width=900, plot_height=850, 
            tools=["pan", "wheel_zoom", "reset","crosshair","box_zoom"])

plot.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 30px 30px 30px 30px'/>
    </div>
    <div>
        <span style='font-size: 10px; color: #224499'>Anchor:</span>
        <span style='font-size: 10px'>@anchor</span>
    </div>
</div>
"""))

# plot settings
plot.scatter('x', 'y', size=5, 
          source=source,
          
          line_alpha=0.3,
          line_color="black")

# this is different from what we learn up to now.
# update method: When slider is changed or when different value from drop down tool is chosen this method is called.
# In this method x and y axis are updated from drop dawn value and year is updated from slider value.
show(plot,notebook_handle=True) 

In [ ]:
from sklearn.manifold import TSNE

pca_50 = PCA(n_components=5)
principalComponents_50 = pca_50.fit_transform(pred)

tsne_50 = TSNE(random_state = 42, n_components=2,verbose=0, perplexity=50, n_iter=300).fit_transform(principalComponents_50)

In [ ]:
# initial source
sourceA = ColumnDataSource(data=dict(
    x = tsne_50[:, 0],
    y = tsne_50[:, 1],
    x_backup = tsne_50[:, 0],
    y_backup = tsne_50[:, 1],
    anchor = data_valid.anchor,
    positive = data_valid.positive,
    negative = data_valid.negative
))

# hover tool
hoverA = HoverTool(tooltips = [("anchor","@anchor"),("positive","@positive"),("negative","@negative")],
                 point_policy="follow_mouse")

# plotting
plotA=figure(title ="TSNE on PCA Visualization",
            plot_width=900, plot_height=850, 
            tools=[hoverA,"crosshair","pan","box_zoom"])
# plot settings
plotA.scatter('x', 'y', size=5, 
          source=sourceA,
          
          line_alpha=0.3,
          line_color="black")

# this is different from what we learn up to now.
# update method: When slider is changed or when different value from drop down tool is chosen this method is called.
# In this method x and y axis are updated from drop dawn value and year is updated from slider value.
show(plotA,notebook_handle=True) 

In [ ]:
# initial source
source = ColumnDataSource(data=dict(
    x = tsne_50[:, 0],
    y = tsne_50[:, 1],
    x_backup = tsne_50[:, 0],
    y_backup = tsne_50[:, 1],
    anchor = data_valid.anchor,
    image = list(map(embeddable_image, data_valid2.anchor))
))

# hover tool
#hover = HoverTool(tooltips = [("anchor","@anchor"),("positive","@positive"),("negative","@negative")],
#                 point_policy="follow_mouse")

# plotting
plot=figure(title ="TSNE on PCA50 Visualization with Anchor Images",
            plot_width=900, plot_height=850, 
            tools=["pan", "wheel_zoom", "reset","crosshair","box_zoom"])

plot.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 30px 30px 30px 30px'/>
    </div>
    <div>
        <span style='font-size: 10px; color: #224499'>Anchor:</span>
        <span style='font-size: 10px'>@anchor</span>
    </div>
</div>
"""))

# plot settings
plot.scatter('x', 'y', size=5, 
          source=source,
          
          line_alpha=0.3,
          line_color="black")

# this is different from what we learn up to now.
# update method: When slider is changed or when different value from drop down tool is chosen this method is called.
# In this method x and y axis are updated from drop dawn value and year is updated from slider value.
show(plot,notebook_handle=True) 